In [8]:
import numpy as np
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import os

# Load datasets
actual_data = pd.read_csv('original_CMKL1.csv')
synthetic_data_ml = pd.read_csv('synthetic_data_RandomSeaerch_Ensemble.csv')
synthetic_data_gan = pd.read_csv('GANs_synthetic_data.csv')

# Combine synthetic datasets for the ML+GAN case
synthetic_data_combined = pd.concat([synthetic_data_ml, synthetic_data_gan])

# Create directory for saving models
if not os.path.exists('Tensor_NEW_senior_saved_model'):
    os.makedirs('Tensor_NEW_senior_saved_model')

# Define TensorFlow models
def create_classification_model(input_shape):
    model = models.Sequential()
    model.add(layers.InputLayer(shape=input_shape))  # Use 'shape' instead of 'input_shape'
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adam(), 
                  loss=losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

def create_regression_model(input_shape):
    model = models.Sequential()
    model.add(layers.InputLayer(shape=input_shape))  # Use 'shape' instead of 'input_shape'
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(2, activation='linear'))
    model.compile(optimizer=optimizers.Adam(), 
                  loss=losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

# Function to create combined dataset with given ratio for each floor
def create_combined_dataset(actual_data, synthetic_data, ratio):
    combined_data = pd.DataFrame()
    floors = actual_data['z'].unique()
    for floor in floors:
        actual_floor_data = actual_data[actual_data['z'] == floor]
        synthetic_floor_data = synthetic_data[synthetic_data['z'] == floor]
        
        n_actual = int(len(actual_floor_data) * ratio)
        n_synthetic = len(actual_floor_data) - n_actual
        
        actual_sample = actual_floor_data.sample(n_actual, random_state=42)
        if len(synthetic_floor_data) < n_synthetic:
            synthetic_sample = synthetic_floor_data.sample(n_synthetic, replace=True, random_state=42)
        else:
            synthetic_sample = synthetic_floor_data.sample(n_synthetic, random_state=42)
        
        combined_floor_data = pd.concat([actual_sample, synthetic_sample])
        combined_data = pd.concat([combined_data, combined_floor_data], ignore_index=True)
        
    return combined_data

# Function to train TensorFlow model
def train_model_with_tensorflow(X_train, y_train, model, epochs=50, batch_size=32):
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    return model, history.history

# Function to calculate Mean Distance Error
def mean_distance_error(y_true, y_pred):
    return np.mean(np.sqrt(np.sum((y_true - y_pred) ** 2, axis=1)))

# Function to evaluate model
def evaluate_model(model, X_test, y_test, is_coordinate_model=False):
    # Remove the unnecessary conversion to NumPy arrays
    # X_test = X_test.values
    # y_test = y_test.values

    # For classification, ensure y_test is in the shape (batch_size, 1)
    if not is_coordinate_model and len(y_test.shape) == 1:
        y_test = np.expand_dims(y_test, axis=-1)
    
    results = model.evaluate(X_test, y_test, verbose=0)
    predictions = model.predict(X_test)

    if is_coordinate_model:
        mean_dist_error = mean_distance_error(y_test, predictions)
        return {
            'Predictions': predictions,
            'Mean Distance Error (meters)': mean_dist_error,
            'Evaluation Results': results
        }
    else:
        mae = mean_absolute_error(y_test, predictions)
        return {
            'Predictions': predictions,
            'Mean Absolute Error': mae,
            'Evaluation Results': results
        }


# Load and test models using the whole actual data
def load_model_and_predict(model_path, test_data, is_coordinate_model=False):
    model = tf.keras.models.load_model(model_path)
    # Convert test data to NumPy array
    X_test = test_data[[f'RSSI{i+1}' for i in range(18)]].values
    y_test = test_data[['z']] if not is_coordinate_model else test_data[['x', 'y']]
    y_test = y_test.values  # Ensure y_test is also a NumPy array
    return evaluate_model(model, X_test, y_test, is_coordinate_model)

# Train and evaluate models
ratios = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
datasets = {
    'actual_ml': synthetic_data_ml,
    'actual_gan': synthetic_data_gan,
    'actual_ml_gan': synthetic_data_combined
}
results = {}

for dataset_name, synthetic_data in datasets.items():
    for ratio in ratios:
        combined_data = create_combined_dataset(actual_data, synthetic_data, ratio)
        
        # Floor classification training and evaluation
        X_train, X_test, y_train, y_test = train_test_split(
            combined_data[[f'RSSI{i+1}' for i in range(18)]],  # 18 RSSI features
            combined_data['z'],  # Floor label
            test_size=0.3, random_state=42
        )
        
        input_shape = (X_train.shape[1],)
        classification_model = create_classification_model(input_shape)
        classification_model, history = train_model_with_tensorflow(X_train, y_train, classification_model)

        # Save the model using the recommended format
        classification_model.save(f'Tensor_NEW_senior_saved_model/{dataset_name}_{ratio}_classification_model.keras')

        # Evaluate model
        classification_results = evaluate_model(classification_model, X_test, y_test)
        results[f'{dataset_name}_{ratio}_classification'] = classification_results
        
        # Coordinate regression training and evaluation
        X_train, X_test, y_train, y_test = train_test_split(
            combined_data[[f'RSSI{i+1}' for i in range(18)]],  # 18 RSSI features
            combined_data[['x', 'y']],  # Coordinate labels
            test_size=0.3, random_state=42
        )
        
        regression_model = create_regression_model(input_shape)
        regression_model, history = train_model_with_tensorflow(X_train, y_train, regression_model)

        # Save the model using the recommended format
        regression_model.save(f'Tensor_NEW_senior_saved_model/{dataset_name}_{ratio}_regression_model.keras')

        # Evaluate model
        regression_results = evaluate_model(regression_model, X_test, y_test, is_coordinate_model=True)
        results[f'{dataset_name}_{ratio}_regression'] = regression_results

# Example usage of the load_and_predict_with_model function
prediction_results = {}
model_directory = 'Tensor_NEW_senior_saved_model'
dataset_names = list(datasets.keys())
ratios = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

# Load and predict using each stored model
for dataset_name in dataset_names:
    for ratio in ratios:
        model_path = f'{model_directory}/{dataset_name}_{ratio}_classification_model.keras'
        key = f'{dataset_name}_{ratio}_classification'
        prediction_results[key] = load_model_and_predict(model_path, actual_data)

        model_path = f'{model_directory}/{dataset_name}_{ratio}_regression_model.keras'
        key = f'{dataset_name}_{ratio}_regression'
        prediction_results[key] = load_model_and_predict(model_path, actual_data, is_coordinate_model=True)

# Print results
for key, metrics in prediction_results.items():
    print(f'Results for {key}:')
    for metric_name, value in metrics.items():
        print(f'  {metric_name}: {value}')

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step
46/46 ━━━━━━━━━━